In [1]:
import os 
import torch
from src.util import plotting_util as plot_util
from src.lib import vertex_labeling
from src.util import data_extraction_util as data_util
from src.util import pairtree_data_extraction_util as pt_util

npz = os.path.join("/Users/divyakoyyalagunta/Desktop/", "P-0035156_IMPACT505.orchard.npz")
tsv = os.path.join("/Users/divyakoyyalagunta/Desktop/", "P-0035156_IMPACT505_clustered_SNVs.tsv")


16:59:51 INFO
NumExpr defaulting to 8 threads.


CUDA GPU: False


In [2]:
primary_site = "lung"
patient_name='P-0035156'
data = pt_util.get_adj_matrices_from_pairtree_results(npz)
tree, _ = data[0] # Use best tree
ref_matrix, var_matrix, unique_sites, idx_to_cluster_label = data_util.get_ref_var_matrices_from_real_data(tsv)
    
idx_to_cluster_label = {k:(" ").join(v.split('_')[0:2]) for k,v in idx_to_cluster_label.items()}
print(idx_to_cluster_label)

print(f"Anatomical sites: {unique_sites}")   
primary_idx = unique_sites.index(primary_site)
p = torch.nn.functional.one_hot(torch.tensor([primary_idx]), num_classes=len(unique_sites)).T
weights = vertex_labeling.Weights(data_fit=1.0, mig=3.0, comig=2.0, seed_site=1.0, reg=2.0, gen_dist=0.0)
print_config = plot_util.PrintConfig(visualize=False, verbose=False, viz_intermeds=False, k_best_trees=3)
vertex_labeling.get_migration_history(tree, ref_matrix, var_matrix, unique_sites, p, idx_to_cluster_label, weights,
                                      print_config, os.path.join("/Users/divyakoyyalagunta/Desktop/"), patient_name,
                                      batch_size=32, max_iter=100)


{0: 'EGFR p.L858R', 1: 'TP53 p.E171*', 2: 'IGF1R p.V1176M', 3: 'TP53 p.E171*'}
Anatomical sites: ['lung', 'bone']


ValueError: Labels in node_idx_to_label should be unique, got: ['EGFR p.L858R', 'TP53 p.E171*', 'IGF1R p.V1176M', 'TP53 p.E171*']